<h1><center>Análisis de Sistemas de Infraestructura <br>
    Taller 2: Machine Learning </center></h1>

<h2> 2. Regresión logística y Redes neuronales.</h2>

 Importar las librerías que usaremos en el taller

In [2]:
#Importar las librerías necesarias

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import time
from sklearn import preprocessing
import tensorflow as tf #Importacion de librerias de trabajo
from tensorflow import keras
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mpl_toolkits.mplot3d import Axes3D
import import_ipynb
import Parte1Taller2#Libreria de nuestro archivo primera parte del taller, para poder importarla de esta forma ambos archivos tienen que estar en una misma ruta (misma carpeta) 

%matplotlib inline

importing Jupyter notebook from Parte1Taller2.ipynb


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


The elapsed time was 0.0
The model performance for training set
--------------------------------------
RMSE is 0.1181009111855882
R2 score is 0.6324965086938636


The model performance for testing set
--------------------------------------
RMSE is 0.1381513927349074
R2 score is -1.851552533562546
The elapsed time was 0.000995635986328125
The model performance for training set
--------------------------------------
RMSE is 0.02049250815658428
R2 score is 0.8012519374302236


The model performance for testing set
--------------------------------------
RMSE is 0.29398717913771466
R2 score is 0.00865374750549286
The elapsed time was 0.0
The model performance for training set
--------------------------------------
RMSE is 0.07052550506097634
R2 score is 0.6593396496909949


The model performance for testing set
--------------------------------------
RMSE is 0.2576336462299111
R2 score is 0.16765601655994988
The elapsed time was 0.0
The model performance for training set
--------------------

<h3> Importar los datos </h3>

In [29]:
#Importamos algunos DataFrame con los que trabajamos en la primera parte del taller 
from Parte1Taller2 import df_GDP, Datos_pilar_infr, lista_nombres_ind

<h3>Divida los países en 4 categorías según los cuartiles del GDP. A cada país asigne un número que
determine la categoría a la que pertenece (i.e. números del 0 al 3).</h3>

In [25]:
longitud=len(df_GDP) # Numero de paises

# Obtenermos Q1, Q2, Q3

mediana=df_GDP.median() 
Q3=df_GDP.quantile(0.75) #Cuartil 3
Q1=df_GDP.quantile(0.25) #Cuartil 1

df_GDP=df_GDP.sort_values(by=0,ascending=False) #Ordenar 

print('Q1 es', Q1)
print('La mediana de los datos es', mediana)
print('Q3 3s', Q3)

La mediana de los datos es 0    0.001233
dtype: float64
Q1 es 0    0.000299
Name: 0.25, dtype: float64
Q3 0    0.010498
Name: 0.75, dtype: float64


In [27]:
#Datos por encima del Q3 - Clasificacion 3

cont=(Q3<df_GDP).sum() #Numero de valores por encima del cuartil Q3

Cuartil_superior=np.zeros(int(cont))
for n in range (int(cont)):
    Cuartil_superior[n]=df_GDP.iloc[n]
    
Cuartil_superior=pd.DataFrame(Cuartil_superior, index=df_GDP[:int(cont)].index)
Cuartil_superior['clasificación']=3

# Datos entre el Q2 y Q3

cont1=(mediana<df_GDP).sum() #Numero de valores entre Q2 y Q3
cont1=cont1-cont

Cuartil_Q3=np.zeros(int(cont1))
for n in range (int(cont1)):
    Cuartil_Q3[n]=df_GDP.iloc[n+int(cont)]

df_GDP_v=df_GDP[int(cont):]
df_GDP_v=df_GDP_v[:int(cont1)]

Cuartil_Q3=pd.DataFrame(Cuartil_Q3, index=df_GDP_v.index)
Cuartil_Q3['clasificación']=2

# Datos entre el Q1 y Q2

cont2=(Q1<df_GDP).sum() #Numero de valores entre Q1 y Q2
cont2=cont2-cont1-cont

Cuartil_Q1=np.zeros(int(cont2))
for n in range (int(cont2)):
    Cuartil_Q1[n]=df_GDP.iloc[n+int(cont1)+int(cont)]

lim=int(cont)+int(cont1)
df_GDP_v=df_GDP[lim:]
df_GDP_v=df_GDP_v[:int(cont2)]

Cuartil_Q1=pd.DataFrame(Cuartil_Q1, index=df_GDP_v.index)
Cuartil_Q1['clasificación']=1

# Datos por debajo del Q1

cont3=(Q1>df_GDP).sum() #Numero de valores por abajo de Q1

Cuartil_inferior=np.zeros(int(cont3))
for n in range (int(cont3)):
    Cuartil_inferior[n]=df_GDP.iloc[n+int(cont1)+int(cont)+int(cont2)]

lim=int(cont)+int(cont1)+int(cont2)
df_GDP_v=df_GDP[lim:]
df_GDP_v=df_GDP_v[:int(cont3)]

Cuartil_inferior=pd.DataFrame(Cuartil_inferior, index=df_GDP_v.index)
Cuartil_inferior['clasificación']=0

In [12]:
# Un solo Dataframe con los datos clasificados

frames = [Cuartil_superior, Cuartil_Q3, Cuartil_Q1, Cuartil_inferior]
datos_GDP_clasificados=pd.concat(frames)
datos_GDP_clasificados.columns=['GDP','clasificacion']
datos_GDP_clasificados

,GDP,clasificacion
Country Name,,
United States,1.000000,3
China,0.667881,3
Japan,0.239369,3
Germany,0.181852,3
South Asia,0.171085,3
...,...,...
Palau,0.000011,0
Marshall Islands,0.000008,0
Kiribati,0.000006,0


<h3> Entrene una red neuronal cuya entrada sean los indicadores contenidos en el Pilar de Infraestructura del GCI y que entregue la categoría a la que pertenece el país. <h3>

In [13]:
data=pd.merge(Datos_pilar_infr, datos_GDP_clasificados, on='Country Name', suffixes=('_left', '_right'))

X = data.iloc[:,0:11].values
Y = data['clasificacion'].values

min_max_scaler = preprocessing.MinMaxScaler()
X_scaler = min_max_scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaler, Y, test_size = 0.2)

print(X_train.shape, X_test.shape, Y_train.shape,Y_test.shape)

(100, 11) (26, 11) (100,) (26,)


In [15]:
# Define a simple sequential model

def create_model():
    
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=(11,)),
        keras.layers.Dense(100, activation='relu', input_shape=(11,)),
        keras.layers.Dense(4, activation='softmax'), #4 neuronas de salida porque tenemos 4 clasificaciones posibles
      ])
    model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    return model

# Create a basic model instance

model = create_model() #Creacion del modelo de red neuronal

# Display the model's architecture

model.summary() #Resum del modelo creado

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 11)                0         
                                                                 
 dense_2 (Dense)             (None, 100)               1200      
                                                                 
 dense_3 (Dense)             (None, 4)                 404       
                                                                 
Total params: 1604 (6.27 KB)
Trainable params: 1604 (6.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
t=time.time()
model.fit(X_train, 
          Y_train,  
          epochs=70,
          validation_data=(X_test,Y_test)); #Entrenamiento del modelo y validacion
elapsed=time.time()-t
elapsed_r=round(elapsed,3)
print('The elapsed time was {}'.format(elapsed_r))

Epoch 1/70
4/4 [==============================] - 2s 158ms/step - loss: 1.2897 - accuracy: 0.3600 - val_loss: 1.2971 - val_accuracy: 0.2692
Epoch 2/70
4/4 [==============================] - 0s 22ms/step - loss: 1.2475 - accuracy: 0.3600 - val_loss: 1.2781 - val_accuracy: 0.2692
Epoch 3/70
4/4 [==============================] - 0s 23ms/step - loss: 1.2131 - accuracy: 0.3600 - val_loss: 1.2722 - val_accuracy: 0.2692
Epoch 4/70
4/4 [==============================] - 0s 21ms/step - loss: 1.1837 - accuracy: 0.3600 - val_loss: 1.2644 - val_accuracy: 0.3077
Epoch 5/70
4/4 [==============================] - 0s 21ms/step - loss: 1.1607 - accuracy: 0.3600 - val_loss: 1.2504 - val_accuracy: 0.3077
Epoch 6/70
4/4 [==============================] - 0s 23ms/step - loss: 1.1387 - accuracy: 0.3600 - val_loss: 1.2406 - val_accuracy: 0.3077
Epoch 7/70
4/4 [==============================] - 0s 22ms/step - loss: 1.1202 - accuracy: 0.3600 - val_loss: 1.2321 - val_accuracy: 0.3462
Epoch 8/70
4/4 [==========

In [17]:
 _,test_acc= model.evaluate(X_test, Y_test, verbose=2)
print('\nTest accuracy:', test_acc)
Y_test

1/1 - 0s - loss: 1.0432 - accuracy: 0.6538 - 45ms/epoch - 45ms/step

Test accuracy: 0.6538461446762085


array([0, 3, 2, 1, 2, 1, 1, 3, 1, 0, 2, 3, 0, 1, 2, 3, 2, 2, 2, 3, 3, 2,
       2, 3, 2, 2], dtype=int64)

In [18]:
predictions = model.predict(X_test) #Prediciones del modelo
predictions[5] #Ejemplo de prediccion
np.argmax(predictions[5])

1/1 [==============================] - 0s 192ms/step


2

<h3> Entrene una hipótesis logística que cumpla la misma función de la red neuronal del literal anterior. <h3>

In [19]:
X_log_regression= data.iloc[:,0:11].values
Y_log_regression = data['clasificacion'].values

min_max_scaler_reg = preprocessing.MinMaxScaler()
X_scaler_log_regression = min_max_scaler.fit_transform(X_log_regression)

X_scaler_log_regression=np.append(X_scaler_log_regression,X_scaler_log_regression**2,axis=1) #Se añaden variables cuadradas
X_log_regression_train, X_log_regression_test, Y_log_regression_train, Y_log_regression_test = train_test_split(X_scaler_log_regression, Y_log_regression, test_size = 0.2, random_state=1) # se separan train y test set
lr = LogisticRegression() #Se genera el modelo logistico
t=time.time()
lr.fit(X_log_regression_train, Y_log_regression_train) #Se entrena el modelo
elapsed=time.time()-t
print('The elapsed time was {}'.format(elapsed))
y_pred = lr.predict(X_log_regression_train) #Se realiza la prediccion del train set
confusion=confusion_matrix(Y_log_regression_train, y_pred)/np.sum(confusion_matrix(Y_log_regression_train, y_pred)) #generacion de la matriz de errores
accuracy=np.trace(confusion) #calculo de la exactitud
print("The model performance for training set")
print("--------------------------------------")
print('Accuracy of {}'.format(accuracy))
y_pred = lr.predict(X_log_regression_test)
confusion=confusion_matrix(Y_log_regression_test, y_pred)/np.sum(confusion_matrix(Y_log_regression_test, y_pred))
accuracy=np.trace(confusion)
print("The model performance for testing set")
print("--------------------------------------")
print('Accuracy of {}'.format(accuracy))

The elapsed time was 0.11352944374084473
The model performance for training set
--------------------------------------
Accuracy of 0.75
The model performance for testing set
--------------------------------------
Accuracy of 0.6153846153846154


In [30]:
predictions1 = lr.predict(X_log_regression_test) #Prediciones del modelo
predictions1[1] #Ejemplo de prediccion

1

In [31]:
7.579/0.11352944374084473

66.75801228534768

<h3> Si solo se aumenta el indicador de calidad en las vías ¿es posible que Colombia cambie de categoría? <h3>

In [32]:
X_2=pd.DataFrame(X_scaler,index=data.index)
X_2.columns = lista_nombres_ind

originales_Col=np.zeros(11)

for n in range (0,11):
    originales_Col[n]=X_2[lista_nombres_ind[n]]['Colombia']

# Numero de fila de Colombia
ind=0
for i in range(0, len(X_2)):
    if X_2['Quality of road infrastructure'].iloc[i]==X_2['Quality of road infrastructure']['Colombia']:
        ind=i
    
valor_calidad_vias_Col=X_2['Quality of road infrastructure']['Colombia']
predictions = model.predict(X_2) #Prediciones  
categoria_prediccion_Col=np.argmax(predictions[ind])
print('De acuerdo con la red neuronal (predicion) Colombia esta en categoria', categoria_prediccion_Col)
print('De acuerdo con la clasificacion real por cuartiles Colombia esta en la categoria', datos_GDP_clasificados['clasificacion']['Colombia'])

4/4 [==============================] - 0s 7ms/step
De acuerdo con la red neuronal (predicion) Colombia esta en categoria 3
De acuerdo con la clasificacion real por cuartiles Colombia esta en la categoria 3


In [34]:
X_2['Quality of road infrastructure'].iloc[ind]=1 #Cambiando este indicador (0 - 1) puede comprobar en esta celda si la categoria varía
predictions = model.predict(X_2) #Prediciones 
predictions[ind] #Predicción de Colombia
np.argmax(predictions[ind])

4/4 [==============================] - 0s 5ms/step


3

### ¿Qué indicador de infraestructura tiene mayor influencia en el GDP de Colombia? –analice la forma en la que cambian las probabilidades entregadas por la red neuronal para cambios marginales en cada indicador.

In [35]:
X_2['Quality of road infrastructure'].iloc[ind]=originales_Col[1] #Regreso a mis datos originales 

#Asumir que cada indicador aumenta en 15%
def aumento_Colombia(indicador):
    X_2[indicador].iloc[ind]=X_2[indicador].iloc[ind]*1.15
    return(model.predict(X_2)[ind])

#Regresar el indicador a su condición original
def regreso(indicador,n):
    X_2[indicador].iloc[ind]=originales_Col[n]

In [23]:
dataset_aum={}

dataset_aum['Normal']=model.predict(X_2)[ind] # Sin aumentar los valores de los indicadores como son las probabilidades

for i in range(0,len(lista_nombres_ind)):
    dataset_aum['Aumentando indicador %d' %i]=aumento_Colombia(lista_nombres_ind[i])
    regreso(lista_nombres_ind[i],i)

pd.DataFrame.from_dict(dataset_aum, orient='index') # Note que las columnas representan probabilidades (probabilidad de que el modelo clasifique 0, 1, 2 o 3). 

4/4 [==============================] - 0s 2ms/step


,0,1,2,3
Normal,0.014567,0.168285,0.326614,0.490533
Aumentando indicador 0,0.013784,0.163055,0.327555,0.495606
Aumentando indicador 1,0.014360,0.165502,0.329669,0.490469
Aumentando indicador 2,0.014513,0.167234,0.327184,0.491069
Aumentando indicador 3,0.014219,0.163212,0.321638,0.500931
Aumentando indicador 4,0.011705,0.125419,0.282346,0.580529
Aumentando indicador 5,0.014007,0.177279,0.327703,0.481010
Aumentando indicador 6,0.012330,0.140882,0.297547,0.549241
Aumentando indicador 7,0.013893,0.175204,0.340523,0.470380
Aumentando indicador 8,0.014653,0.189468,0.340748,0.455131
